# <a href="https://emoji.gg/sticker/72567-doro"><img src="https://cdn3.emoji.gg/stickers/72567-doro.png" width="32px" height="32px" alt="Doro"></a> Welcome to the Hugging Face File Uploader! 

# Welcome to the Hugging Face File Uploader

Use this notebook's widget to easily upload files to your Hugging Face repositories.

## Getting Started

- Install Libraries: Run the first code cell.
- Log In: Run the notebook_login() cell with a Hugging Face write token.
- Launch Uploader: Run the final cell to display the uploader widget.

## Uploading Files

- Repo Info: Fill in the destination repository details.
- File Selection: Point to a local directory and select your files.
- Upload: Click the upload button to start the process.

## Key Features
- Fast Uploads: Automatically uses hf_transfer to speed up large file transfers.
- Live Progress: Monitor upload status and speed in the output log.
- Simple Interface: All controls are contained within a single, easy-to-use widget.


**Community & Support:**

*   **GitHub:** [HuggingFace\_Backup Repository on GitHub](https://github.com/Ktiseos-Nyx/HuggingFace_Backup) (for the latest version, updates, bug reports, and contributions)
*   **Discord:**
    *   [Ktiseos Nyx AI/ML Discord](https://discord.gg/HhBSvM9gBY)
    *   [Earth & Dusk Media](https://discord.gg/5t2kYxt7An)

This uploader is designed to simplify the process of getting your files onto the Hugging Face Hub. We hope you find it useful!

 #  <a href="https://emoji.gg/sticker/72567-doro"><img src="https://cdn3.emoji.gg/stickers/72567-doro.png" width="32px" height="32px" alt="Doro"></a> Install Dependencies

In [1]:
# Cell 1: Install Required Python Packages
# -----------------------------------------------------------------------------
print("Installing/updating required packages...")
!pip install -q huggingface_hub
!pip install -q ipywidgets  
!pip install -q hf_transfer
print("Package installation/update process complete.")


Installing/updating required packages...
Package installation/update process complete.


# ✨  <a href="https://emoji.gg/sticker/72567-doro"><img src="https://cdn3.emoji.gg/stickers/72567-doro.png" width="32px" height="32px" alt="Doro"></a> Connecting to Hugging Face: Authentication

## 🔑 Hugging Face Login
You need a write-enabled token to upload files.

### Instructions:

- Create a Token: Go [here](https://huggingface.co/settings/tokens), click "New token", and give it the write role.
- Copy the Token: Copy the new token to your clipboard.
- Run Login Cell: Execute the code cell below, paste your token when prompted, and press Enter.


Troubleshooting: If you get an error, the most common problem is that your token is missing write permissions. Double-check the token's role on the Hugging Face website.

In [1]:
# Cell 2: Hugging Face Authentication Setup
# -----------------------------------------------------------------------------
from huggingface_hub import notebook_login
import os
notebook_login();



# 🚀  <a href="https://emoji.gg/sticker/72567-doro"><img src="https://cdn3.emoji.gg/stickers/72567-doro.png" width="32px" height="32px" alt="Doro"></a> Using the Hugging Face File Uploader

## Uploader Checklist
Follow these steps to upload your files.

1. Fill in Repository Details
- Owner (your username/org)
- Repo Name
- Repo Type (model, dataset, etc.)

2. Select Local Files
- Set the Source Directory to your local folder path.
- Click the 🔄 List Files button.
- Select your desired files from the list that appears.

3. Review Upload Settings (Optional)
- Add a Commit Message to describe your changes.
- Choose whether to Create a Pull Request for this upload.

4. Start the Upload
- Click the ⬆️ Upload Selected Files button and monitor the output.

In [4]:
# --- Essential Imports for the Uploader ---
import glob
import os
import time
from pathlib import Path
import math # For math.log and math.floor in _format_size

from huggingface_hub import HfApi
from ipywidgets import (Text, Dropdown, Button, SelectMultiple, VBox, HBox,
                        Output, Layout, Checkbox, HTML, Textarea, Label,
                        FloatProgress)
from IPython.display import display, clear_output

# Attempt to enable hf_transfer.
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

class HuggingFaceUploader:
    """
    A Jupyter widget-based tool to upload files to the Hugging Face Hub.
    """

    def __init__(self):
        self.api = HfApi()
        self.file_types = [
            # AI Model Files 🤖
            ('SafeTensors', 'safetensors'), ('PyTorch Models', 'pt'), ('PyTorch Legacy', 'pth'),
            ('ONNX Models', 'onnx'), ('TensorFlow Models', 'pb'), ('Keras Models', 'h5'),
            # Checkpoint Files 🎯
            ('Checkpoints', 'ckpt'), ('Binary Files', 'bin'),
            # Config & Data Files 📝
            ('JSON Files', 'json'), ('YAML Files', 'yaml'), ('YAML Alt', 'yml'),
            ('Text Files', 'txt'), ('CSV Files', 'csv'), ('Pickle Files', 'pkl'),
            # Image Files 🎨
            ('PNG Images', 'png'), ('JPEG Images', 'jpg'), ('JPEG Alt', 'jpeg'),
            ('WebP Images', 'webp'), ('GIF Images', 'gif'),
            # Archive Files 📦
            ('ZIP Archives', 'zip'), ('TAR Files', 'tar'), ('GZ Archives', 'gz')
        ]
        self.current_directory = os.getcwd()
        self.hf_transfer_active = self._check_hf_transfer_availability()
        self._create_widgets()
        self._bind_events()
        self._update_files(None) # Initial file list update

    def _check_hf_transfer_availability(self):
        if os.environ.get("HF_HUB_ENABLE_HF_TRANSFER") == "1":
            try:
                import hf_transfer
                return True
            except ImportError:
                return False
        return False

    def _create_widgets(self):
        # --- Repository Info ---
        self.repo_info_html = HTML(value="<b>📚 Repository Details</b>")
        self.org_name_text = Text(placeholder='Organization or Username', description='Owner:', style={'description_width': 'initial'})
        self.repo_name_text = Text(placeholder='Repository Name', description='Repo:', style={'description_width': 'initial'})
        self.repo_type_dropdown = Dropdown(options=['model', 'dataset', 'space'], value='model', description='Repo Type:', style={'description_width': 'initial'})
        self.repo_folder_text = Text(placeholder='Optional: e.g., models/v1', description='Remote Folder:', style={'description_width': 'initial', "flex": "1 1 auto"}, layout=Layout(width='auto'))

        # --- File Selection ---
        self.file_section_html = HTML(value="<b>🗂️ File Selection & Source</b>")
        self.file_type_dropdown = Dropdown(options=self.file_types, value='safetensors', description='File Type:', style={'description_width': 'initial'})
        self.sort_by_dropdown = Dropdown(options=['name', 'date'], value='name', description='Sort By:', style={'description_width': 'initial'})
        self.recursive_search_checkbox = Checkbox(value=False, description='Search Subdirectories', indent=False)

        self.directory_label = Label(value="Source Directory:", layout=Layout(width='auto'))
        self.directory_text = Text(value=self.current_directory, description="", style={'description_width': '0px'}, layout=Layout(width="auto", flex='1 1 auto'))
        self.directory_update_btn = Button(description='🔄 List Files', button_style='info', tooltip='Change source directory and refresh file list', layout=Layout(width='auto'))

        # --- Commit Details ---
        self.commit_section_html = HTML(value="<b>💭 Commit Details</b>")
        self.commit_msg_textarea = Textarea(value="Upload via HuggingFaceUploader Widget 🤗", placeholder='Enter your commit message (optional)', description='Message:', style={'description_width': 'initial'}, layout=Layout(width='98%', height='60px'))

        # --- Upload Settings ---
        self.upload_section_html = HTML(value="<b>🚀 Upload Settings</b>")
        self.create_pr_checkbox = Checkbox(value=False, description='Create Pull Request', indent=False)
        self.clear_after_checkbox = Checkbox(value=True, description='Clear output after upload', indent=False)

        # --- Action Buttons ---
        self.upload_button = Button(description='⬆️ Upload Selected Files', button_style='success', tooltip='Start upload process', layout=Layout(width='auto', height='auto'))
        self.clear_output_button = Button(description='🧹 Clear Output Log', button_style='warning', tooltip='Clear the output log area', layout=Layout(width='auto'))

        # --- File Picker & Output ---
        self.file_picker_selectmultiple = SelectMultiple(options=[], description='Files:', layout=Layout(width="98%", height="200px"), style={'description_width': 'initial'})
        self.output_area = Output(layout=Layout(padding='10px', border='1px solid #ccc', margin_top='10px', width='98%', max_height='400px', overflow_y='auto'))

        # --- Progress Display Area ---
        self.current_file_label = Label(value="N/A")
        self.file_count_label = Label(value="File 0/0")
        self.progress_bar = FloatProgress(value=0, min=0, max=100, description='Overall:', bar_style='info', layout=Layout(width='85%'))
        self.progress_percent_label = Label(value="0%")

        self.progress_display_box = VBox([
            HBox([Label("Current File:", layout=Layout(width='100px')), self.current_file_label]),
            HBox([Label("File Count:", layout=Layout(width='100px')), self.file_count_label]),
            HBox([self.progress_bar, self.progress_percent_label], layout=Layout(align_items='center'))
        ], layout=Layout(visibility='hidden', margin='10px 0', padding='10px', border='1px solid #ddd', width='98%'))

    def _bind_events(self):
        self.directory_update_btn.on_click(self._update_directory_and_files)
        self.upload_button.on_click(self._upload_files_handler)
        self.clear_output_button.on_click(lambda _: self.output_area.clear_output(wait=True))
        self.file_type_dropdown.observe(self._update_files, names='value')
        self.sort_by_dropdown.observe(self._update_files, names='value')
        self.recursive_search_checkbox.observe(self._update_files, names='value') # NEW BINDING

    def _update_directory_and_files(self, _):
        new_dir = self.directory_text.value.strip()
        if not new_dir:
            with self.output_area:
                clear_output(wait=True); print(f"📂 Current directory remains: {self.current_directory}")
            self._update_files(None)
            return

        if os.path.isdir(new_dir):
            self.current_directory = os.path.abspath(new_dir)
            self.directory_text.value = self.current_directory
            self._update_files(None)
        else:
            with self.output_area:
                clear_output(wait=True); print(f"❌ Invalid Directory: {new_dir}")

    def _update_files(self, _):
        file_extension = self.file_type_dropdown.value
        self.output_area.clear_output(wait=True)
        try:
            # ENHANCEMENT: Use glob '**/ ' for recursive search
            glob_prefix = '**/' if self.recursive_search_checkbox.value else ''
            glob_pattern = f"{glob_prefix}*.{file_extension}"
            
            if not os.path.isdir(self.current_directory):
                with self.output_area: print(f"⚠️ Source directory '{self.current_directory}' is not valid.")
                self.file_picker_selectmultiple.options = []
                return

            # Use rglob for simplicity if recursive
            source_path = Path(self.current_directory)
            found_paths = list(source_path.rglob(f'*.{file_extension}')) if self.recursive_search_checkbox.value else list(source_path.glob(f'*.{file_extension}'))
            
            valid_files_info = []
            for p in found_paths:
                if p.is_symlink() or not p.is_file(): continue
                sort_key = p.stat().st_mtime if self.sort_by_dropdown.value == 'date' else p.name.lower()
                valid_files_info.append((str(p), sort_key))

            if self.sort_by_dropdown.value == 'date':
                valid_files_info.sort(key=lambda item: item[1], reverse=True)
            else:
                valid_files_info.sort(key=lambda item: item[1])
            
            # Display relative paths in the picker for clarity, but store absolute paths
            # self.file_picker_selectmultiple.options now a list of (display_name, value)
            display_options = []
            for abs_path_str, _ in valid_files_info:
                display_name = os.path.relpath(abs_path_str, self.current_directory)
                display_options.append((display_name, abs_path_str))
            
            self.file_picker_selectmultiple.options = display_options
            
            with self.output_area:
                if not display_options:
                     print(f"🤷 No '.{file_extension}' files found in '{self.current_directory}'.")
                else:
                    print(f"✨ Found {len(display_options)} '.{file_extension}' files. Select files to upload.")

        except Exception as e:
            with self.output_area:
                clear_output(wait=True); print(f"❌ Error listing files: {e}"); traceback.print_exc(file=self.output_area)

    def _format_size(self, size_bytes):
        if size_bytes < 0: return "Invalid size"
        if size_bytes == 0: return "0 B"
        units = ("B", "KB", "MB", "GB", "TB", "PB", "EB")
        # CORRECTED: Ensure direct calls to math module, not Path.math
        i = math.floor(math.log(size_bytes, 1024)) if size_bytes > 0 else 0
        if i >= len(units): i = len(units) - 1
        s = round(size_bytes / (1024 ** i), 2)
        return f"{s} {units[i]}"

    def _print_file_info(self, file_path_str, index, total_files):
        file_path = Path(file_path_str)
        try:
            file_size = file_path.stat().st_size
            self.output_area.append_stdout(f"📦 Uploading {index}/{total_files}: {file_path.name} ({self._format_size(file_size)})\n")
        except FileNotFoundError:
            self.output_area.append_stdout(f"⚠️ File not found: {file_path_str}\n")

    def _upload_files_handler(self, _):
        org_or_user = self.org_name_text.value.strip()
        repo_name = self.repo_name_text.value.strip()

        if not org_or_user or not repo_name:
            with self.output_area: clear_output(wait=True); print("❗ Please fill in 'Owner' and 'Repo Name'.")
            return

        repo_id = f"{org_or_user}/{repo_name}"
        selected_file_paths = list(self.file_picker_selectmultiple.value)

        if not selected_file_paths:
            with self.output_area: clear_output(wait=True); print("📝 Nothing selected for upload.")
            return

        self.output_area.clear_output(wait=True)
        self.output_area.append_stdout(f"🎯 Preparing to upload to: https://huggingface.co/{repo_id}\n")
        if self.hf_transfer_active: self.output_area.append_stdout("🚀 HF_TRANSFER is enabled.\n")
        else: self.output_area.append_stdout("ℹ️ For faster uploads, run `%pip install -q hf_transfer` and restart kernel.\n")

        self.progress_display_box.layout.visibility = 'visible'
        self.progress_bar.value = 0
        self.progress_percent_label.value = "0%"
        self.current_file_label.value = "Initializing..."
        
        total_files = len(selected_file_paths)
        self.file_count_label.value = f"File 0/{total_files}"
        
        repo_type = self.repo_type_dropdown.value
        repo_folder_prefix = self.repo_folder_text.value.strip().replace('\\', '/')
        base_commit_msg = self.commit_msg_textarea.value or "Upload via HuggingFaceUploader Widget 🤗"

        success_count = 0
        for idx, local_file_path_str in enumerate(selected_file_paths, 1):
            try:
                current_file_path = Path(local_file_path_str)
                self.current_file_label.value = current_file_path.name
                self.file_count_label.value = f"File {idx}/{total_files}"
                self._print_file_info(local_file_path_str, idx, total_files)
                
                start_time = time.time()
                
                if not current_file_path.exists():
                    self.output_area.append_stdout(f"❌ SKIPPED: File '{current_file_path.name}' not found.\n")
                    continue
                
                # This logic correctly handles recursive uploads
                path_in_repo_base = os.path.relpath(current_file_path, self.current_directory).replace('\\', '/')
                path_in_repo = f"{repo_folder_prefix}/{path_in_repo_base}" if repo_folder_prefix.strip('/') else path_in_repo_base
                
                commit_message_for_file = f"{base_commit_msg} ({current_file_path.name})"

                response_url = self.api.upload_file(
                    path_or_fileobj=str(current_file_path),
                    path_in_repo=path_in_repo,
                    repo_id=repo_id,
                    repo_type=repo_type,
                    create_pr=self.create_pr_checkbox.value,
                    commit_message=commit_message_for_file,
                )
                duration = time.time() - start_time
                self.output_area.append_stdout(f"✅ Uploaded '{current_file_path.name}' to '{path_in_repo}' in {duration:.1f}s.\n")
                self.output_area.append_stdout(f"   View at: {response_url}\n")
                success_count += 1

            except Exception as e:
                self.output_area.append_stdout(f"❌ Error uploading {current_file_path.name}: {e}\n")
                import traceback
                with self.output_area: traceback.print_exc()
                self.output_area.append_stdout("\n")
            finally:
                percentage = int((idx / total_files) * 100)
                self.progress_bar.value = percentage
                self.progress_percent_label.value = f"{percentage}%"

        self.output_area.append_stdout(f"\n✨ Upload complete. {success_count}/{total_files} files processed. ✨\n")
        if self.create_pr_checkbox.value and success_count > 0:
            self.output_area.append_stdout(f"🎉 View Pull Request: https://huggingface.co/{repo_id}/pulls\n")
        elif success_count > 0 :
            repo_tree_url = f"https://huggingface.co/{repo_id}/tree/main/{repo_folder_prefix.strip('/')}".rstrip('/')
            self.output_area.append_stdout(f"🎉 View files at: {repo_tree_url}\n")

        self.current_file_label.value = "Completed."
        if self.clear_after_checkbox.value:
            time.sleep(5)
            self.output_area.clear_output(wait=True)
            self.progress_display_box.layout.visibility = 'hidden'

    def display(self):
        repo_box = HBox([self.org_name_text, self.repo_name_text, self.repo_type_dropdown], layout=Layout(flex_flow='wrap', justify_content='space-between'))
        repo_folder_box = HBox([self.repo_folder_text], layout=Layout(width='100%'))
        dir_select_box = HBox([self.directory_label, self.directory_text, self.directory_update_btn], layout=Layout(width='100%', align_items='center'))
        file_opts_box = HBox([self.file_type_dropdown, self.sort_by_dropdown, self.recursive_search_checkbox], layout=Layout(flex_flow='wrap', justify_content='space-between', align_items='center'))
        upload_opts_box = HBox([self.create_pr_checkbox, self.clear_after_checkbox], layout=Layout(margin='5px 0'))
        action_buttons_box = HBox([self.upload_button, self.clear_output_button], layout=Layout(margin='10px 0 0 0', spacing='10px'))

        main_layout = VBox([
            self.repo_info_html, repo_box, repo_folder_box,
            HTML("<hr>"),
            self.file_section_html, file_opts_box, dir_select_box,
            self.file_picker_selectmultiple,
            HTML("<hr>"),
            self.commit_section_html, self.commit_msg_textarea,
            HTML("<hr>"),
            self.upload_section_html, upload_opts_box,
            action_buttons_box,
            self.progress_display_box,
            self.output_area
        ], layout=Layout(width='700px', padding='10px', border='1px solid lightgray'))
        
        display(main_layout)

# 🚀  <a href="https://emoji.gg/sticker/72567-doro"><img src="https://cdn3.emoji.gg/stickers/72567-doro.png" width="32px" height="32px" alt="Doro"></a> Uploader Widget! 

**Run the next cell to initiate the uploader widget!**


In [5]:
# Uploader Widget Code
print("🚀 Initializing Hugging Face Uploader...")
uploader = HuggingFaceUploader()
uploader.display()
print("✅ Uploader interface is ready. You can now select files and upload.")

🚀 Initializing Hugging Face Uploader...


✅ Uploader interface is ready. You can now select files and upload.
